In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Date
0,World,3566111,2422.0,248285.0,139.0,1154014.0,2163812,50046.0,457.0,31.9,0.0,0.0,2020-05-03 23:08:49.961491
1,USA,1188122,0.0,68598.0,0.0,178263.0,941261,16139.0,3589.0,207.0,7196740.0,21742.0,2020-05-03 23:08:49.961491
2,Spain,247122,0.0,25264.0,0.0,148558.0,73300,2386.0,5285.0,540.0,1932455.0,41332.0,2020-05-03 23:08:49.961491
3,Italy,210717,0.0,28884.0,0.0,81654.0,100179,1501.0,3485.0,478.0,2153772.0,35622.0,2020-05-03 23:08:49.961491
4,UK,186599,0.0,28446.0,0.0,0.0,157809,1559.0,2749.0,419.0,1206405.0,17771.0,2020-05-03 23:08:49.961491
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Western Sahara,6,0.0,0.0,0.0,5.0,1,0.0,10.0,0.0,0.0,0.0,2020-05-03 23:08:49.961491
211,Anguilla,3,0.0,0.0,0.0,3.0,0,0.0,200.0,0.0,0.0,0.0,2020-05-03 23:08:49.961491
212,Comoros,3,0.0,0.0,0.0,0.0,3,0.0,3.0,0.0,0.0,0.0,2020-05-03 23:08:49.961491
213,Saint Pierre Miquelon,1,0.0,0.0,0.0,0.0,1,0.0,173.0,0.0,0.0,0.0,2020-05-03 23:08:49.961491


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,...,Tests/ 1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Spain,247122,0.0,25264.0,0.0,148558.0,73300,2386.0,5285.0,540.0,...,41332.0,2020-05-03 23:08:49.961491,-3.647550,40.244487,173822.0,0.145344,0.102233,0.124301,0.012708,Last 8 columns are added by Roaya
2,Italy,210717,0.0,28884.0,0.0,81654.0,100179,1501.0,3485.0,478.0,...,35622.0,2020-05-03 23:08:49.961491,12.070013,42.796626,110538.0,0.261304,0.137075,0.079046,0.010835,Last 8 columns are added by Roaya
0,USA,1188122,0.0,68598.0,0.0,178263.0,941261,16139.0,3589.0,207.0,...,21742.0,2020-05-03 23:08:49.961491,-97.922211,39.381266,246861.0,0.277881,0.057736,0.176531,0.010192,Last 8 columns are added by Roaya
4,France,168693,0.0,24895.0,0.0,50784.0,93014,3819.0,2584.0,381.0,...,16856.0,2020-05-03 23:08:49.961491,2.618787,47.824905,75679.0,0.328955,0.147576,0.054118,0.007987,Last 8 columns are added by Roaya
5,Germany,165664,0.0,6866.0,0.0,130600.0,28198,1979.0,1977.0,82.0,...,30400.0,2020-05-03 23:08:49.961491,10.385781,51.106982,137466.0,0.049947,0.041445,0.098302,0.004074,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Rwanda,259,0.0,0.0,0.0,124.0,135,0.0,20.0,0.0,...,2652.0,2020-05-03 23:08:49.961491,29.919885,-1.990338,124.0,0.000000,0.000000,0.000089,0.000000,Last 8 columns are added by Roaya
153,Eritrea,39,0.0,0.0,0.0,26.0,13,0.0,11.0,0.0,...,0.0,2020-05-03 23:08:49.961491,38.846170,15.361866,26.0,0.000000,0.000000,0.000019,0.000000,Last 8 columns are added by Roaya
120,Vietnam,271,0.0,0.0,0.0,219.0,52,8.0,3.0,0.0,...,2681.0,2020-05-03 23:08:49.961491,106.299147,16.646017,219.0,0.000000,0.000000,0.000157,0.000000,Last 8 columns are added by Roaya
150,Macao,45,0.0,0.0,0.0,39.0,6,1.0,69.0,0.0,...,0.0,2020-05-03 23:08:49.961491,113.509321,22.223117,39.0,0.000000,0.000000,0.000028,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"Country":"USA","TotalCases":1188122,"NewCases":0.0,"TotalDeaths":68598.0,"NewDeaths":0.0,"TotalRecovered":178263.0,"ActiveCases":941261,"CriticalCases":16139.0,"Tot\\u00a0Cases\\/1M pop":3589.0,"Deaths\\/1M pop":207.0,"TotalTests":7196740.0,"Tests\\/ 1M pop":21742.0,"Date":1588547329961,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":246861.0,"ClosedDeathRate":0.2778810748,"CurrentDeathRate":0.0577364951,"Weight":0.1765309092,"DeathRateXWeight":0.010192276,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":247122,"NewCases":0.0,"TotalDeaths":25264.0,"NewDeaths":0.0,"TotalRecovered":148558.0,"ActiveCases":73300,"CriticalCases":2386.0,"Tot\\u00a0Cases\\/1M pop":5285.0,"Deaths\\/1M pop":540.0,"TotalTests":1932455.0,"Tests\\/ 1M pop":41332.0,"Date":1588547329961,"Longitude":-3.64755047,"Latitude":40.24448698,"ClosedCases":173822.0,"ClosedDeathRate":0.1453440876,"CurrentDeathRate":0.1022329052,"Weight":0.1243005404,"DeathRateXWeight":0.0127076054,"not